## Импортирование зависимостей

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as sts
import numpy as np

C:\Users\damir\AppData\Local\Temp\ipykernel_15160\3901000916.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Конфигурация графиков

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['font.size'] = 14

## Загрузка и просмотр датасета 

In [3]:
path: str = r'data.csv'
data = pd.read_csv(path, index_col='Customer ID')

In [4]:
data.head()

,Gender,Age,City,Membership Type,Total Spend,Items Purchased,Average Rating,Discount Applied,Days Since Last Purchase,Satisfaction Level
Customer ID,,,,,,,,,,
101,Female,29,New York,Gold,1120.20,14,4.6,True,25,Satisfied
102,Male,34,Los Angeles,Silver,780.50,11,4.1,False,18,Neutral
103,Female,43,Chicago,Bronze,510.75,9,3.4,True,42,Unsatisfied
104,Male,30,San Francisco,Gold,1480.30,19,4.7,False,12,Satisfied
105,Male,27,Miami,Silver,720.40,13,4.0,True,55,Unsatisfied


In [5]:
data = data[~data['Satisfaction Level'].isna()]

## Рассмотрим гипотезы:
$H_0$ - не существует различия между событиями
\
$H_1$ - существует различие между событиями

In [6]:
cond = data.Gender == 'Male'
male = data[cond]
female = data[~cond]

In [7]:
def test(male: pd.DataFrame, female: pd.DataFrame, feature: str, md: str) -> None:
    a = .05

    if  md == 'm':
        _, p_value = sts.mannwhitneyu(male[feature], female[feature])
    elif md == 'd':
        _, p_value = sts.ks_2samp(male[feature], female[feature])
    else:
        print('Wrong input!')
        return None
    
    print(f'{p_value=}')
    
    if p_value > a:
        print("Не существует различия между событиями")
    else:
        print("Существует различие между событиями")

## Пункт №1 (Непрерывный случай, оценка медианы)

In [8]:
male['Total Spend'].median(), female['Total Spend'].median()

(800.9, 505.75)

In [9]:
test(male, female, 'Total Spend', 'm')

p_value=5.424545127316686e-19
Существует различие между событиями


## Пункт №2 (Непрерывный случай, оценка распределения)

In [10]:
test(male, female, 'Total Spend', 'd')

p_value=2.879145633100441e-36
Существует различие между событиями


## Пункт №3 (Дискретный случай, оценка медианы)

In [11]:
male['Items Purchased'].median(), female['Items Purchased'].median()

(13.0, 9.0)

In [12]:
test(male, female, 'Items Purchased', 'm')

p_value=9.687135147271277e-18
Существует различие между событиями


## Пункт №4 (Дискретный случай, оценка распредления)


In [13]:
test(male, female, 'Items Purchased', 'd')

p_value=1.357631805149866e-22
Существует различие между событиями


## Проверка релевантным тестом

In [14]:
a = .05
for i in 'cd':
    for j in 'md':
        if i == 'c' and j == 'm':
            print('Непрерывный случай, оценка медианы')
            print('Возьмем альтернативой U-тесту тест Муда')
            p_value = sts.median_test(male['Total Spend'], female['Total Spend'])[1]
        elif i == 'c' and j == 'd':
            print('Непрерывный случай, оценка распределения')
            print('Возьмем альтернативой тесту Колмогорова-Смирнова U-test')
            _, p_value = sts.mannwhitneyu(male['Total Spend'], female['Total Spend'])
        elif i =='d' and j == 'm':
            print('Дискретный случай, оценка медианы')
            print('Возьмем альтернативой U-тесту тест Муда')
            p_value = sts.median_test(male['Items Purchased'], female['Items Purchased'])[1]
        elif i == 'd' and j == 'd':
            print('Дискретный случай, оценка медианы')
            print('Возьмем альтернативой тесту Колмогорова-Смирнова U-test')
            p_value = sts.mannwhitneyu(male['Items Purchased'], female['Items Purchased'])[1]
        else:
            print('Wrong input!')

        print(f'{p_value=}')
        if p_value > a:
            print("Не существует различия между событиями")
        else:
            print("Существует различие между событиями")
        print()

Непрерывный случай, оценка медианы
Возьмем альтернативой U-тесту тест Муда
p_value=4.4196773130945145e-08
Существует различие между событиями

Непрерывный случай, оценка распределения
Возьмем альтернативой тесту Колмогорова-Смирнова U-test
p_value=5.424545127316686e-19
Существует различие между событиями

Дискретный случай, оценка медианы
Возьмем альтернативой U-тесту тест Муда
p_value=0.0011048837865400792
Существует различие между событиями

Дискретный случай, оценка медианы
Возьмем альтернативой тесту Колмогорова-Смирнова U-test
p_value=9.687135147271277e-18
Существует различие между событиями



## Пункт №5(Бутстрап)

In [15]:
def bootstrap(male: pd.DataFrame, female: pd.DataFrame, feature: str, md: str, n: int):
    n = int(1e4)
    count = 0
    
    for i in range(n):
        m = np.random.choice(male[feature], size=len(male), replace=True)
        f = np.random.choice(female[feature], size=len(female), replace=True)
        
        if  md == 'm':
            _, p_value = sts.mannwhitneyu(m, f)
        elif md == 'd':
            _, p_value = sts.ks_2samp(m, f)
        else:
            print('Wrong input!')
            return None
        
        if p_value < 0.05:
            count += 1
    
    avg = count / n
    print(f'Average count of p_values < 0.05: {avg}')

In [16]:
bootstrap(male, female, 'Total Spend', 'm', 10000)
print('Бутстрап лучше, т.к. неизвестна дисперсия выборок')

Average count of p_values < 0.05: 1.0
Бутстрап лучше, т.к. неизвестна дисперсия выборок


In [17]:
bootstrap(male, female, 'Total Spend', 'd', 10000)
print('Бутстрап лучше, т.к. неизвестна дисперсия выборок')

Average count of p_values < 0.05: 1.0
Бутстрап лучше, т.к. неизвестна дисперсия выборок


In [18]:
bootstrap(male, female, 'Items Purchased', 'm', 10000)
print('Отсутсвие преимуществ среди методов')

Average count of p_values < 0.05: 1.0
Отсутсвие преимуществ среди методов


In [19]:
bootstrap(male, female, 'Items Purchased', 'd', 10000)
print('Отсутсвие преимуществ среди методов')

Average count of p_values < 0.05: 1.0
Отсутсвие преимуществ среди методов
